In [1]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install catboost
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
train = pd.read_csv('./drive/My Drive/TGIH/Train.csv')
test = pd.read_csv('./drive/My Drive/TGIH/Test.csv')
sub = pd.read_csv('./drive/My Drive/TGIH/Sample Submission.csv')

In [5]:
train.shape
test.shape
sub.shape

(284780, 8)

(122049, 7)

(122049, 1)

In [6]:
sub.head()
train.head()
train.tail()
test.head()
test.tail()

,UnitPrice
0,100
1,100
2,100
3,100
4,100


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
284775,3408,1706,1081,6,2011-03-01 12:07:00,1.65,17965.0,35
284776,13982,2184,2947,16,2011-10-13 15:08:00,0.65,16793.0,35
284777,6641,667,3632,4,2011-05-16 12:59:00,8.50,14755.0,35
284778,13820,1576,3089,48,2011-10-11 14:18:00,3.45,14866.0,35
284779,11042,2603,1156,2,2011-08-22 13:30:00,0.75,17585.0,35


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
0,3139,1709,1079,3,2011-02-22 15:22:00,16161.0,35
1,617,510,3457,1,2010-12-08 12:46:00,17341.0,35
2,14653,604,694,36,2011-10-25 13:53:00,15158.0,35
3,8634,1478,3473,2,2011-06-27 12:38:00,16033.0,35
4,15546,3216,871,1,2011-11-06 16:14:00,15351.0,35


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
122044,7813,2487,1289,12,2011-06-09 09:44:00,15214.0,35
122045,8694,3069,217,16,2011-06-28 15:47:00,16200.0,35
122046,915,1419,3474,6,2010-12-13 12:26:00,15555.0,35
122047,8156,1631,1434,1,2011-06-16 12:45:00,15907.0,35
122048,12020,2088,706,3,2011-09-12 13:50:00,16360.0,35


In [7]:
# Missing Values
train.isnull().sum().sum()
test.isnull().sum().sum()

0

0

In [8]:
train.describe()
test.describe()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
count,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000
mean,9955.394083,1573.386807,2023.955573,12.028468,3.451216,15288.224278,32.823846
std,5551.519138,843.604991,1089.812078,295.111588,78.399541,1714.336529,6.743602
min,0.000000,0.000000,0.000000,-80995.000000,0.000000,12346.000000,0.000000
25%,5069.000000,939.000000,1141.000000,2.000000,1.250000,13953.000000,35.000000
50%,10310.000000,1521.000000,1987.000000,5.000000,1.950000,15152.000000,35.000000
75%,14657.000000,2106.000000,2945.000000,12.000000,3.750000,16794.000000,35.000000
max,22188.000000,3683.000000,3895.000000,80995.000000,38970.000000,18287.000000,36.000000


,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country
count,122049.000000,122049.000000,122049.000000,122049.000000,122049.000000,122049.000000
mean,9940.419692,1572.955788,2026.412744,12.137920,15286.445256,32.836795
std,5554.116349,845.106082,1087.697388,54.311646,1711.887591,6.726816
min,0.000000,0.000000,0.000000,-9360.000000,12347.000000,0.000000
25%,5018.000000,937.000000,1147.000000,2.000000,13953.000000,35.000000
50%,10311.000000,1523.000000,1987.000000,5.000000,15152.000000,35.000000
75%,14635.000000,2106.000000,2945.000000,12.000000,16782.000000,35.000000
max,22189.000000,3683.000000,3895.000000,4800.000000,18287.000000,36.000000


In [9]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    284780 non-null  int64  
 1   StockCode    284780 non-null  int64  
 2   Description  284780 non-null  int64  
 3   Quantity     284780 non-null  int64  
 4   InvoiceDate  284780 non-null  object 
 5   UnitPrice    284780 non-null  float64
 6   CustomerID   284780 non-null  float64
 7   Country      284780 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 17.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122049 entries, 0 to 122048
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    122049 non-null  int64  
 1   StockCode    122049 non-null  int64  
 2   Description  122049 non-null  int64  
 3   Quantity     122049 non-null  int64  
 4   InvoiceDate  122049 non-

In [10]:
#there are duplicates between test and train, for those rows we can directly use train target values
#storing the target values of common rows in test and train
temp1=''
for i in test.columns:
    temp1=temp1+train[i].astype(str)
train['ALL']=temp1
temp2=''
for i in test.columns:
    temp2=temp2+test[i].astype(str)
test['ALL']=temp2
encoding=train.groupby('ALL')['UnitPrice'].median()
y_from_train=test['ALL'].map(encoding)

In [11]:
#no. of rows which were directly replaced
len(test)-y_from_train.isnull().sum()

2212

In [12]:
train=train.drop('ALL',1)
test=test.drop('ALL',1)

In [13]:
df = train.append(test)
df.shape

(406829, 8)

In [14]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'], dtype='object')

In [15]:
df.drop(columns=['CustomerID','InvoiceNo'],axis=1,inplace=True)

In [16]:
df.shape
df['InvoiceDate']= pd.to_datetime(df['InvoiceDate'])
df.info()

(406829, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 122048
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   StockCode    406829 non-null  int64         
 1   Description  406829 non-null  int64         
 2   Quantity     406829 non-null  int64         
 3   InvoiceDate  406829 non-null  datetime64[ns]
 4   UnitPrice    284780 non-null  float64       
 5   Country      406829 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 21.7 MB


In [17]:
df['hour'] = df['InvoiceDate'].apply(lambda x: x.hour)
df['minute'] = df['InvoiceDate'].apply(lambda x: x.minute)
df['day'] = df['InvoiceDate'].apply(lambda x: x.day)
df['month'] = df['InvoiceDate'].apply(lambda x: x.month)
df['year'] = df['InvoiceDate'].apply(lambda x: x.year)
df.drop('InvoiceDate',axis=1,inplace=True)
df.head()

,StockCode,Description,Quantity,UnitPrice,Country,hour,minute,day,month,year
0,1583,144,3,3.75,35,16,54,6,5,2011
1,1300,3682,6,1.95,35,7,35,11,5,2011
2,2178,1939,4,5.95,35,13,20,20,11,2011
3,2115,2983,1,0.83,35,12,7,22,11,2011
4,1210,2886,12,1.65,35,9,0,6,5,2011


In [18]:
for col in ['StockCode','Description','Quantity','Country','hour','minute','day','month','year']:
  print(df[col].nunique())

3684
3896
436
37
15
60
31
12
2


In [19]:
features = [col for col in df.columns if col!='UnitPrice']
len(features)

9

In [20]:
train = df.head(len(train))
test = df.tail(len(test))
train.shape,test.shape

((284780, 10), (122049, 10))

In [22]:
test.drop('UnitPrice',axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
train.shape,test.shape

((284780, 10), (122049, 9))

In [34]:
X = train.drop('UnitPrice',axis=1)
y = train['UnitPrice']

In [27]:
train.isnull().sum()

StockCode      0
Description    0
Quantity       0
UnitPrice      0
Country        0
hour           0
minute         0
day            0
month          0
year           0
dtype: int64

In [50]:
from sklearn.metrics import mean_squared_error

In [51]:
from sklearn.model_selection import KFold
err = [] 
y_pred_tot_lgbm = np.zeros((len(test), ))


fold = KFold(n_splits=10, shuffle=True, random_state=2020)
i = 1

for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    m = LGBMRegressor(n_estimators=5000)
    m.fit(x_train, y_train,eval_set=[(x_val, y_val)],early_stopping_rounds=100,verbose=0)
    pred_val = m.predict(x_val)
    print(i, " err_lgm: ", np.sqrt(mean_squared_error(y_val,pred_val)))
    err.append(np.sqrt(mean_squared_error(y_val,pred_val)))
    y_pred_tot_lgbm+= m.predict(test)
    i = i + 1
y_pred_tot_lgbm=y_pred_tot_lgbm/10
sum(err)/10

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

1  err_lgm:  48.67068030998703


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

2  err_lgm:  17.863492274950183


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

...

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

9  err_lgm:  27.306785570224097


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

10  err_lgm:  27.753326593620532


44.17985916328608

In [30]:
import pandas as pd

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [31]:
train = clean_dataset(train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [33]:
test = clean_dataset(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [52]:
y_pred_tot_lgbm

array([3.55504909, 3.20233435, 2.36965257, ..., 2.70751615, 4.57509168,
       3.53462469])

In [54]:
# writing prediction of lgbm to a csv file
sub['UnitPrice']=y_pred_tot_lgbm
sub['UnitPrice']=y_from_train.fillna(sub['UnitPrice'])
sub.to_csv('lgbm_kfold_v2.csv',index=False)

In [57]:
err = [] 
y_pred_tot_xgb = np.zeros((len(test), ))


fold = KFold(n_splits=10, shuffle=True, random_state=2020)
i = 1

for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    m = XGBRegressor(n_estimators=1001)
    m.fit(x_train, y_train,eval_set=[(x_val, y_val)],early_stopping_rounds=100,verbose=0)
    pred_val = m.predict(x_val)
    print("\n",i, " err_xgb: ", np.sqrt(mean_squared_error(y_val,pred_val)))
    err.append(np.sqrt(mean_squared_error(y_val,pred_val)))
    y_pred_tot_xgb+= m.predict(test)
    i = i + 1
y_pred_tot_xgb=y_pred_tot_xgb/10
sum(err)/10

[17:09:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1001,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


 1  err_xgb:  47.54473961129083
[17:09:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1001,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


 2  err_xgb:  16.802660370295644
[17:09:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1001,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


 9  err_xgb:  27.75217553994324
[17:12:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1001,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


 10  err_xgb:  32.18743297679157


44.32486346769875

In [58]:
# writing prediction of lgbm to a csv file
sub['UnitPrice']=y_pred_tot_xgb
sub['UnitPrice']=y_from_train.fillna(sub['UnitPrice'])
sub.to_csv('xgb_kfold_v2.csv',index=False)

In [59]:

err = [] 
y_pred_tot_catb = np.zeros((len(test), ))


fold = KFold(n_splits=10, shuffle=True, random_state=2020)
i = 1

for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    m = CatBoostRegressor(n_estimators=5000)
    m.fit(x_train, y_train,eval_set=[(x_val, y_val)],early_stopping_rounds=50,verbose=0)
    pred_val = m.predict(x_val)
    print("\n",i, " err_xgb: ", np.sqrt(mean_squared_error(y_val,pred_val)))
    err.append(np.sqrt(mean_squared_error(y_val,pred_val)))
    y_pred_tot_catb+= m.predict(test)
    i = i + 1
y_pred_tot_catb=y_pred_tot_catb/10
sum(err)/10


 1  err_xgb:  48.21811489454106



 2  err_xgb:  18.561087125221995



 3  err_xgb:  230.32016087253393



 4  err_xgb:  12.126787747543998



 5  err_xgb:  21.3361488745414


...


 7  err_xgb:  24.269193474988306



 8  err_xgb:  14.715762401840886



 9  err_xgb:  29.566438493210104



 10  err_xgb:  23.159163543731804


44.348997782143

In [60]:
# writing prediction of lgbm to a csv file
sub['UnitPrice']=y_pred_tot_catb
sub['UnitPrice']=y_from_train.fillna(sub['UnitPrice'])
sub.to_csv('catb_kfold_v2.csv',index=False)